> __Purpose:__ This notebook will implement our MAML functions and test the episodic dataloader structure. Once everything is running, either this NB or a copy will do HPO for the MAML parameters. We will include the MOE layer for now (although placement is probably quite poor), and we will stick with the CNN-MLP architecture since the CNN-LSTM architecture was not performing as well (the latter architecture was probably not fit sufficiently in its HPO).

In [1]:
CODE_DIR = "C:\\Users\\kdmen\\Repos\\pers-gest-cls\\"
DATA_DIR = "C:\\Users\\kdmen\\Box\\Yamagami Lab\\Data\\Meta_Gesture_Project\\filtered_datasets\\"
RUN_DIR  = "C:\\Users\\kdmen\\Repos\\pers-gest-cls\\system\\results\\"

from pathlib import Path
import os
# env -> Path objects
#CODE_DIR = Path(os.environ.get("CODE_DIR", "./")).resolve()
#DATA_DIR = Path(os.environ.get("DATA_DIR", "./")).resolve()
#RUN_DIR = Path(os.environ.get("RUN_DIR", "./")).resolve()
print(f"CODE_DIR: {CODE_DIR}")
print(f"DATA_DIR: {DATA_DIR}")
print(f"RUN_DIR: {RUN_DIR}")
# TODO: Update these for the appropriate saving. Don't save into the git repo on PROJECTS
#######################################################
# === SAVING (to SCRATCH) ===
#results_save_dir = RUN_DIR.parent / "runs" / f"{timestamp}_MOE"        # /scratch/my13/kai/runs/<timestamp>_MOE
#models_save_dir  = RUN_DIR.parent / "models" / "MOE" / f"{timestamp}_MOE"
results_save_dir = RUN_DIR
models_save_dir  = RUN_DIR
# make sure they exist
#results_save_dir.mkdir(parents=True, exist_ok=True)
#models_save_dir.mkdir(parents=True, exist_ok=True)

CODE_DIR: C:\Users\kdmen\Repos\pers-gest-cls\
DATA_DIR: C:\Users\kdmen\Box\Yamagami Lab\Data\Meta_Gesture_Project\filtered_datasets\
RUN_DIR: C:\Users\kdmen\Repos\pers-gest-cls\system\results\


In [2]:
current_directory = os.getcwd()
print(f"The current working directory is: {current_directory}")

The current working directory is: c:\Users\kdmen\Repos\pers-gest-cls\system\MAML_MOE


In [3]:
import sys

# Get the path to the 'pers-gest-cls' directory
# Since you are in .../system/MAML_MOE, we go up two levels
root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

if root_path not in sys.path:
    sys.path.append(root_path)

#current_directory = os.getcwd()
#print(f"The current working directory is: {current_directory}")

In [4]:
N_TRIALS = 1
FIXED_SEED = 42

import copy, json, time#, joblib, sys
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import optuna
#from optuna.samplers import TPESampler
#from optuna.pruners import MedianPruner
#from optuna.storages import JournalStorage, JournalFileBackend
from optuna.storages.journal import JournalStorage, JournalFileBackend
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# === LOADING (from SCRATCH data bucket) ===
# adjust these to where you staged each file under $DATA_DIR
## These are now inputs to the config directly... --> These are actually the NOTS specific versions...
###########################
user_split_json_filepath = CODE_DIR + "system\\fixed_user_splits\\4kfcv_splits_shared_test.json"
def apply_fold_to_config(config, all_splits, fold_idx):
    """Mutates config in-place to set train/val/test PIDs for the given fold."""
    split = all_splits[fold_idx]
    config["train_PIDs"] = split["train"]
    config["val_PIDs"]   = split["val"]
    config["test_PIDs"]  = split["test"]
    config["num_pretrain_users"] = len(config["train_PIDs"])
    config["num_testft_users"] = len(config["val_PIDs"])
###########################

from system.MAML_MOE.multimodal_data_processing import *  # Needed for load_multimodal_dataloaders()
from system.MAML_MOE.mamlpp import *
from system.MAML_MOE.maml_multimodal_dataloaders import *
from system.MAML_MOE.MOE_CNN_LSTM import *

timestamp = datetime.now().strftime("%Y%m%d_%H%M")

In [5]:
import torch
# Assuming MultimodalCNNLSTMMOE is imported from your system modules
# from system.MAML_MOE.mamlpp import MultimodalCNNLSTMMOE 

config = dict()
config["device"] = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
config["feature_engr"] = "None"
config["time_steps"] = 1 
config["sequence_length"] = 64 
config["num_train_gesture_trials"] = 9
config["num_ft_gesture_trials"] = 1
config["padding"] = 0 
config["use_batch_norm"] = False 
config["timestamp"] = timestamp
config["dropout"] = 0.0  # Lowest dropout for debugging
config["num_classes"] = 10
config["use_earlystopping"] = True
config["verbose"] = False
config["num_total_users"] = 32
config["train_gesture_range"] = [1, 2, 3, 4, 5, 6, 7, 8, 9]
config["valtest_gesture_range"] = [2, 3, 4, 5, 6, 7, 8, 9, 10]

config["NOTS"] = False
if config["NOTS"] == False:
    config["emg_imu_pkl_full_path"] = 'C:\\Users\\kdmen\\Box\\Yamagami Lab\\Data\\Meta_Gesture_Project\\filtered_datasets\\metadata_IMU_EMG_allgestures_allusers.pkl'
    config["pwmd_xlsx_filepath"] = "C:\\Users\\kdmen\\Repos\\pers-gest-cls\\dataset\\Biosignal gesture questionnaire for participants with disabilities.xlsx"
    config["pwoutmd_xlsx_filepath"] = "C:\\Users\\kdmen\\Repos\\pers-gest-cls\\dataset\\Biosignal gesture questionnaire for participants without disabilities.xlsx"
    config["dfs_save_path"] = "C:\\Users\\kdmen\\Repos\\pers-gest-cls\\dataset\\meta-learning-sup-que-ds\\"
    config["dfs_load_path"] = "C:\\Users\\kdmen\\Repos\\pers-gest-cls\\dataset\\meta-learning-sup-que-ds\\"
    config["saved_df_timestamp"] = '20250917_1217'
    config["user_split_json_filepath"] = "C:\\Users\\kdmen\\Repos\\pers-gest-cls\\system\\fixed_user_splits\\4kfcv_splits_shared_test.json"
    config["results_save_dir"] = f"C:\\Users\\kdmen\\Repos\\pers-gest-cls\\system\\results\\local_{timestamp}"
    config["models_save_dir"] = f"C:\\Users\\kdmen\\Repos\\pers-gest-cls\\system\\models\\local_{timestamp}"
elif config["NOTS"] == True:
    config["user_split_json_filepath"] = user_split_json_filepath
    config["results_save_dir"] = results_save_dir
    config["models_save_dir"] = models_save_dir
    config["emg_imu_pkl_full_path"] = f"{CODE_DIR}//dataset//filtered_datasets//metadata_IMU_EMG_allgestures_allusers.pkl" 
    config["pwmd_xlsx_filepath"] = f"{CODE_DIR}//dataset//Biosignal gesture questionnaire for participants with disabilities.xlsx"
    config["pwoutmd_xlsx_filepath"] = f"{CODE_DIR}//dataset//Biosignal gesture questionnaire for participants without disabilities.xlsx"
    config["dfs_save_path"] = f"{CODE_DIR}/dataset//"
    config["dfs_load_path"] = f"{CODE_DIR}/dataset/meta-learning-sup-que-ds//"

# ----- Model layout hyperparams (Optimized for speed) -----
config["num_experts"] = 2
config["use_shared_expert"] = True
config["expert_architecture"] = "linear" 
config["top_k"] = 1
config["gate_type"] = "context_only"
config["mixture_mode"] = 'logits'
config["return_aux"] = True 

# NEW MULTIMODAL
config["groupnorm_num_groups"] = 4
config["emg_base_cnn_filters"] = 16
config["imu_base_cnn_filters"] = 16
config['emg_stride'] = 1 
config['imu_stride'] = 1 
config["cnn_kernel_size"] = 3
config["imu_cnn_layers"] = 1
config["emg_cnn_layers"] = 1
config["use_film_x_demo"] = False
config["use_imu"] = True 
config["use_demographics"] = True 
config["context_emb_dim"] = 4
config["context_pool_type"] = 'mean'
config["demo_emb_dim"] = 4

config["multimodal"] = True
config['emg_in_ch'] = 16
config['imu_in_ch'] = 72
config['demo_in_dim'] = 12
config['num_epochs'] = 2  # Reduced drastically for quick debugging

config["use_lstm"] = True
if config["use_lstm"]:
    config["lstm_hidden"] = 32
    config["lstm_layers"] = 1
    config["use_GlobalAvgPooling"] = True

# MOE Hyperparams
config["label_smooth"] = 0.1

# Pretraining optim
config["weight_decay"] = 1e-4
config["optimizer"] = "adam"
config["lr_scheduler_factor"] = 0.1
config["lr_scheduler_patience"] = 6
config["earlystopping_patience"] = 8
config["earlystopping_min_delta"] = 0.005

# MAML SPECIFIC
config["meta_learning"] = True
config["n_way"] = 10
config["k_shot"] = 1
config["q_query"] = 2  # Lowered for faster debugging
config["meta_batchsize"] = 4
config["episodes_per_epoch_train"] = 10  # Very small for rapid testing
config["num_workers"] = 0  # 0 is often safer/easier to debug on Windows locally
config["maml_inner_steps"] = 1

# MAML Optimization Order
config["maml_opt_order"] = "first" 
config["maml_first_order_to_second_order_epoch"] = 1000000 

# MAML Multi-Step Loss
config["use_maml_msl"] = False
config["maml_msl_num_epochs"] = 0

# LSLR Logic
config["maml_use_lslr"] = False
config["learning_rate"] = 0.001
config["maml_alpha_init"] = 0.01
config["maml_alpha_init_eval"] = 0.01

config["enable_inner_loop_optimizable_bn_params"] = False
config["maml_inner_steps_eval"] = 3
config["use_cosine_outer_lr"] = False
config["use_lslr_at_eval"] = False

# ----- Build model -----
model = MultimodalCNNLSTMMOE(config)
device = config["device"]
model.to(device)

MultimodalCNNLSTMMOE(
  (emg_encoder): Sequential(
    (0): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): GroupNorm(4, 16, eps=1e-05, affine=True)
    (2): ReLU()
    (3): Dropout(p=0.0, inplace=False)
  )
  (imu_encoder): Sequential(
    (0): Conv1d(72, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): GroupNorm(4, 16, eps=1e-05, affine=True)
    (2): ReLU()
    (3): Dropout(p=0.0, inplace=False)
  )
  (demo_encoder): DemographicsEncoder(
    (net): Sequential(
      (0): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
      (1): Linear(in_features=12, out_features=16, bias=True)
      (2): GELU(approximate='none')
      (3): Linear(in_features=16, out_features=4, bias=True)
    )
  )
  (lstm): LSTM(32, 32, batch_first=True, bidirectional=True)
  (context_projector): ContextEncoder(
    (projector): Sequential(
      (0): Linear(in_features=64, out_features=4, bias=True)
      (1): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
      (2): GELU(

In [6]:
with open(user_split_json_filepath, "r") as f:
    ALL_SPLITS = json.load(f)

apply_fold_to_config(config, ALL_SPLITS, 0)

In [7]:
# ---- Pretraining data & training ----
# train support, train query, val support (ft), val query (novel test), test support, test query
#train_loader, val_loader, ft_loader, novel_test_loader, test_support_dl, test_query_dl = load_multimodal_data_loaders(config, load_existing_dfs=True)
episodic_train_loader, episodic_val_loader, episodic_test_loader = load_multimodal_data_loaders(config, load_existing_dfs=True)

In [8]:
#num_episodes = len(episodic_val_loader)
#print(f"Total episodes in val_loader: {num_episodes}")

# TypeError: object of type 'FixedOneShotPerUserIterable' has no len()

In [9]:
unique_users = set()

for batch in episodic_val_loader:
    print(f"Type of batch: {type(batch)}")
    
    if isinstance(batch, dict):
        print("Batch is a dictionary! Keys are:", batch.keys())
        # If user_id is in the batch directly:
        user = batch.get('user_id')
        print(f"User ID found: {user}")
        unique_users.add(user)
        
    elif isinstance(batch, list):
        print(f"Batch is a list of length {len(batch)}")
        # If it's a list, the first item is likely your episode dict
        sample = batch[0]
        if isinstance(sample, dict):
             print("First item in list is a dict. Keys:", sample.keys())
             unique_users.add(sample['user_id'])
    break # Just check the first one

Type of batch: <class 'dict'>
Batch is a dictionary! Keys are: dict_keys(['support', 'query', 'user_id', 'label_map', 'classes_global'])
User ID found: 0


In [10]:
count = 0
unique_users = set()

for batch in episodic_val_loader:
    count += 1
    # If the loader collates everything into one dict:
    if 'user_id' in batch:
        u_id = batch['user_id']
        # If it's a single ID (string or int)
        if isinstance(u_id, (str, int)):
            unique_users.add(u_id)
        # If it's a list/tensor of IDs for the whole batch
        else:
            for u in u_id:
                # Convert back from tensor if necessary
                val = u.item() if torch.is_tensor(u) else u
                unique_users.add(val)

print(f"Total iterations: {count}")
print(f"Unique users: {unique_users}")

Total iterations: 4
Unique users: {0, 8, 16, 24}


In [11]:
# Grab the first meta-batch
first_batch = next(iter(episodic_val_loader))

# If your loader returns a list of episodes (common in MAML++):
if isinstance(first_batch, list):
    meta_batch_size = len(first_batch)
    # Check the first episode in that batch
    sample_episode = first_batch[0] 
else:
    # If it returns a single dictionary representing a batch of tasks
    sample_episode = first_batch
    print(sample_episode.keys())
    print(sample_episode['support'].keys())
    # Usually the first dimension of a tensor in the dict is the batch size
    #meta_batch_size = sample_episode['support'].shape[0]
    # # It's 1
    meta_batch_size = 1

print(f"Meta-batch size (tasks per iteration): {meta_batch_size}")

dict_keys(['support', 'query', 'user_id', 'label_map', 'classes_global'])
dict_keys(['emg', 'imu', 'demo', 'labels', 'PIDs'])
Meta-batch size (tasks per iteration): 1


In [12]:
# --- To see ALL users across the entire loader ---
all_val_users = set()
for batch in episodic_val_loader:
    if isinstance(batch, list):
        for episode in batch:
            all_val_users.add(episode['user_id'])
    else:
        # Handle if it's a tensor or a single ID
        u_id = batch['user_id']
        if torch.is_tensor(u_id):
            all_val_users.update(u_id.tolist())
        else:
            all_val_users.add(u_id)

print(f"All unique users in val_loader: {sorted(list(all_val_users))}")

All unique users in val_loader: [0, 8, 16, 24]


In [13]:
# Do the meta pretraining
pretrained_model, pretrain_res_dict = mamlpp_pretrain(model, config, episodic_train_loader, episodic_val_loader=episodic_val_loader)
best_val_acc = pretrain_res_dict["best_val_acc"]
best_state   = pretrain_res_dict["best_state"]


MAML++ Pretraining: Epoch 1 of 2
--- Episode 0/10 | Target Meta-Batch Size: 4 ---
Meta batchsize hit on ep 4! Parameters updating!
Train completed in 1.66s
Train loss/acc: 2.3024, 10.00%
step / ep: 1 / 1
Meta eval user_id: 0
User: 0 | Support Labels (first 5): tensor([0, 1, 2, 3, 4])
First 5 values in first sample: tensor([-0.3034, -0.2770, -0.2505, -0.2291, -0.1930])
step / ep: 2 / 2
Meta eval user_id: 8
User: 8 | Support Labels (first 5): tensor([0, 1, 2, 3, 4])
First 5 values in first sample: tensor([-0.0597, -0.0578, -0.0536,  0.0230,  0.1603])
step / ep: 3 / 3
Meta eval user_id: 16
User: 16 | Support Labels (first 5): tensor([0, 1, 2, 3, 4])
First 5 values in first sample: tensor([-0.0949, -0.0600, -0.0609, -0.0748, -0.0701])
step / ep: 4 / 4
Meta eval user_id: 24
User: 24 | Support Labels (first 5): tensor([0, 1, 2, 3, 4])
First 5 values in first sample: tensor([-0.0002,  0.0014, -0.0015, -0.0014, -0.0005])
Val completed in 1.11s
Val loss/acc: 2.3036, 10.00%
Epoch completed in 2.

In [14]:
user_loaders = make_user_loaders_from_dataloaders(episodic_val_loader, episodic_test_loader, config)


In [15]:
type(user_loaders)

tuple

In [16]:
user_loaders

({0: <torch.utils.data.dataloader.DataLoader at 0x1c11693b8e0>,
  8: <torch.utils.data.dataloader.DataLoader at 0x1c17dbab190>,
  16: <torch.utils.data.dataloader.DataLoader at 0x1c118047eb0>,
  24: <torch.utils.data.dataloader.DataLoader at 0x1c117fdfaf0>},
 {1: <torch.utils.data.dataloader.DataLoader at 0x1c16e8320e0>,
  9: <torch.utils.data.dataloader.DataLoader at 0x1c16e831450>,
  17: <torch.utils.data.dataloader.DataLoader at 0x1c16e832740>,
  25: <torch.utils.data.dataloader.DataLoader at 0x1c16e830250>})

In [17]:
user_accs = []
val_dls = user_loaders[0]
#test_dls = user_loaders[1]
#for pid, (user_val_epi_dl, user_test_epi_dl) in user_loaders.items():
for user_id, user_val_dl in val_dls.items():
    if user_val_dl is None:
        raise ValueError("user_val_dl is None, preventing maml_finetune_and_eval...")
        continue

    # TODO: user_val_dl here is apparently an integer...
    # TODO: Do I need to add some code so it uses alpha_eval and such, like what happens in my maml code?
    val_metrics = meta_evaluate(model, user_val_dl, config, mamlpp_adapt_and_eval)
    final_user_val_loss, final_user_val_acc = val_metrics["loss"], val_metrics["acc"]
    user_accs.append(final_user_val_acc)
    print(f"Final user{user_id} loss: {final_user_val_acc*100:.2f}%")

mean_acc = float(np.mean(user_accs)) if len(user_accs) > 0 else float("nan")

step / ep: 1 / 1
Meta eval user_id: 0
User: 0 | Support Labels (first 5): tensor([0, 1, 2, 3, 4])
First 5 values in first sample: tensor([-0.3034, -0.2770, -0.2505, -0.2291, -0.1930])
Final user0 loss: 10.00%
step / ep: 1 / 1
Meta eval user_id: 8
User: 8 | Support Labels (first 5): tensor([0, 1, 2, 3, 4])
First 5 values in first sample: tensor([-0.0597, -0.0578, -0.0536,  0.0230,  0.1603])
Final user8 loss: 11.11%
step / ep: 1 / 1
Meta eval user_id: 16
User: 16 | Support Labels (first 5): tensor([0, 1, 2, 3, 4])
First 5 values in first sample: tensor([-0.0949, -0.0600, -0.0609, -0.0748, -0.0701])
Final user16 loss: 10.00%
step / ep: 1 / 1
Meta eval user_id: 24
User: 24 | Support Labels (first 5): tensor([0, 1, 2, 3, 4])
First 5 values in first sample: tensor([-0.0002,  0.0014, -0.0015, -0.0014, -0.0005])
Final user24 loss: 10.00%
